In [ ]:
# import all required libraries
import sys, os
import numpy as np
import pandas as pd
import random
from random import shuffle, choice
import time
import os
import glob
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from random import shuffle, choice
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# define a function to build MLP for the trait data.    
def create_mlp(traitstrain, regularizer=None):
  model = Sequential()
  # first layer, remember to remove bias if you are intercalating with batch normalization. ReLu is the activation (nonlinear) function. L1 regularizes the high-dimensional input with many correlated features and is robust to outliers.
  model.add(Dense(150, use_bias=False, input_dim=traitstrain.shape[1], activation="relu", kernel_regularizer=regularizers.l1(0.001)))
  # batch normalization.
  model.add(BatchNormalization())
  # second layer.
  model.add(Dense(150, use_bias=False, activation="relu"))
  model.add(BatchNormalization())
  # third layer.
  model.add(Dense(50, activation="relu"))
  return model

# define a function to build a CNN for the SNP data. 
def create_cnn(xtest, regularizer=None):
  # obtain the input dimensions.
  inputShape = (xtest.shape[1], xtest.shape[2])
  inputs = Input(shape=inputShape)
  x = inputs
  # first convolutional layer, remember to remove bias if you are intercalating with batch normalization.
  x = Conv1D(250, kernel_size=3, activation='relu', use_bias=False, input_shape=(xtest.shape[1], xtest.shape[2]))(x)
  # batch normalization.
  x = BatchNormalization()(x)
  # second layer.
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  # third layer.
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  # pool the CNN outputs.
  x = MaxPooling1D(pool_size=3)(x)
  # flatten in a single vector.
  x = Flatten()(x)
  # this part is similar to the MLP, a fully connected neural network. We intercalated with dropout to reduce overfitting.
  x = Dense(125, activation='relu')(x)
  # dropout.
  x = Dropout(0.5)(x)
  # second layer of the fully connected neural network.
  x = Dense(125, activation='relu')(x)
  x = Dropout(0.5)(x)
  # third layer of the fully connected neural network. This one matches the number of nodes coming out of the MLP.
  x = Dense(50, kernel_regularizer=regularizer)(x)
  x = Activation("relu")(x)
  # Construct the CNN
  model = Model(inputs, x)
  # Return the CNN
  return model

# define a function to combine the outputs of the MLP and the CNN.
# this was obtained from: https://towardsdatascience.com/neural-networks-ensemble-33f33bea7df3
class LinearW(Layer):
    def __init__(self):
        super(LinearW, self).__init__()    
    def build(self, input_shape):
        self.W = self.add_weight(name='name',
                    shape=(1,1,len(input_shape)),
                    initializer='uniform',
                    dtype=tf.float32,
                    trainable=True)
    def call(self, inputs):
        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim
        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)

In [ ]:
## define variables that will be used to train all networks.
# size of the minibatches containing simulations are passed through the network in each epoch.
batch_size = 250
# number of training iterations (epochs) for the SNP only and the combined networks.
epochs = 100
# number of training iterations (epochs) for the traits only networks.
epochs_traits = 500
# number of scenarios being classified.
num_classes = 3

# load the traits simulated under the BM model for the 3 scenarios. 
traits_BM = []
traits_BM = np.loadtxt("./traits/traits_BM.txt").reshape(30000,-1,100)
# transform into a NumPy array. 
traits_BM = np.array(traits_BM)

# standard scale the continuous (BM) traits
scalers = {}
for i in range(traits_BM.shape[2]):
    scalers[i] = StandardScaler(copy=False)
    traits_BM[:, :, i] = scalers[i].fit_transform(traits_BM[:, :, i]) 

# load the SNPs simulated for the 3 scenarios. 
u1 = np.load("./trainingSims/Model_1sp.npz",mmap_mode='r')
u2 = np.load("./trainingSims/Model_2sp.npz",mmap_mode='r')
u3 = np.load("./trainingSims/Model_3sp.npz",mmap_mode='r')

# combine the loaded SNPs in a single NumPy array.
X=np.concatenate((u1['Model_1sp'],u2['Model_2sp'],u3['Model_3sp']),axis=0)

# transform SNP major alleles in -1 and minor in 1.
for arr,array in enumerate(X):
    for idx,row in enumerate(array):
        if np.count_nonzero(row) > len(row)/2:
            X[arr][idx][X[arr][idx] == 1] = -1
            X[arr][idx][X[arr][idx] == 0] = 1
        else:
            X[arr][idx][X[arr][idx] == 0] = -1

# create a label vector in the same order as the simulations.
y=[0 for i in range(len(u1['Model_1sp']))]
y.extend([1 for i in range(len(u2['Model_2sp']))])
y.extend([2 for i in range(len(u3['Model_3sp']))])
y = np.array(y)

# make sure labels, SNP and traits matrices all have the same length.
print (len(y), len(X), len(traits_BM))

# create a list with the same length of the SNP matrices.
shf = list(range(len(X)))
# shuffle the list
shuffle(shf)

# apply the same shuffle to the labels.
y = y[shf]
# apply the same shuffle to the SNP matrices.
X = X[shf]
# apply the same shuffle to the traits matrices.
traits_BM = traits_BM[shf]

# separate the first 75% of labels, SNP and traits matrices as training set. The last 25% are assigned to the test set.
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_BM_train, traits_BM_test = traits_BM[int(len(y)*.25):], traits_BM[:int(len(y)*.25)]

# convert labels to a categorical matrix of binary values (0 or 1). The number of rows is the length of the input vector (number of simulations) and the number of columns is the number of classes (3 scenarios).
ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
# reshape the traits matrices to input them into the MLP
traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_BM_train)
cnn = create_cnn(xtest)

In [ ]:
################################################################################################################################################
#100 BM, 1K SNPs
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model1.layers: print(layer.get_config(), layer.get_weights())

model1.save(filepath='./Trained_Models/Trained_Comb_Model_100BM_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1K SNPs
################################################################################################################################################

xCNN = Dense(num_classes, activation="softmax")(cnn.output)

model2 = Model(inputs=cnn.input, outputs=xCNN)

opt = SGD(learning_rate=0.001)

model2.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model2.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model2.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model2.layers: print(layer.get_config(), layer.get_weights())

model2.save(filepath='./Trained_Models/Trained_CNN_Model_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#100 BM
################################################################################################################################################

mlp = create_mlp(traits_BM_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

opt = SGD(learning_rate=0.001)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_BM_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_BM_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
#print(model.layers[3].get_weights()[0])

model3.save(filepath='./Trained_Models/Trained_Traits_Model_100BM.acc.mod')

In [ ]:
################################################################################################################################################
#100 BM, 50 SNPs
################################################################################################################################################

xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model4 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model4.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model4.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model4.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model4.layers: print(layer.get_config(), layer.get_weights())
model4.save(filepath='./Trained_Models/Trained_Comb_Model_100BM_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 SNPs
################################################################################################################################################
xCNN = Dense(num_classes, activation="softmax")(cnn.output)

model5 = Model(inputs=cnn.input, outputs=xCNN)

model5.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model5.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model5.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model5.layers: print(layer.get_config(), layer.get_weights())
model5.save(filepath='./Trained_Models/Trained_CNN_Model_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 BM, 50 SNPs
################################################################################################################################################

traits_BM50=traits_BM[:,0:50,:]
traits_BM_train, traits_BM_test = traits_BM50[int(len(y)*.25):], traits_BM50[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_BM_train)
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50BM_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 BM
################################################################################################################################################
mlp = create_mlp(traits_BM_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()         
model3.fit(traits_BM_train, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(traits_BM_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_50BM.acc.mod')

In [ ]:
################################################################################################################################################
#50 BM, 20 SNPs
################################################################################################################################################

#X=np.concatenate((u1['simModel1'],u2['simModel2'],u4['simModel4'],u6['simModel6']),axis=0)
#X = X[shf]

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50BM_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#20 SNPs
################################################################################################################################################
xCNN = Dense(num_classes, activation="softmax")(cnn.output)

model7 = Model(inputs=cnn.input, outputs=xCNN)

model7.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model7.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model7.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model7.layers: print(layer.get_config(), layer.get_weights())
model7.save(filepath='./Trained_Models/Trained_CNN_Model_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#10 BM
################################################################################################################################################
traits_BM10=traits_BM[:,0:10,:]
traits_BM_train, traits_BM_test = traits_BM10[int(len(y)*.25):], traits_BM10[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_BM_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_BM_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_BM_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_10BM.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 10 BM
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

traits_BM10=traits_BM[:,0:10,:]
traits_BM_train, traits_BM_test = traits_BM10[int(len(y)*.25):], traits_BM10[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))
mlp = create_mlp(traits_BM_train)

combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10BM_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 10 BM
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
#xtest=xtest[:,0:20,:]
#xtrain=xtrain[:,0:20,:]
traits_BM10=traits_BM[:,0:10,:]
traits_BM_train, traits_BM_test = traits_BM10[int(len(y)*.25):], traits_BM10[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))
mlp = create_mlp(traits_BM_train)
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10BM_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 50 BM
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_BM50=traits_BM[:,0:50,:]
traits_BM_train, traits_BM_test = traits_BM50[int(len(y)*.25):], traits_BM50[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
cnn = create_cnn(xtest)
mlp = create_mlp(traits_BM_train)
# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50BM_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50SNPS, 10 BM
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_BM10=traits_BM[:,0:10,:]
traits_BM_train, traits_BM_test = traits_BM10[int(len(y)*.25):], traits_BM10[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_BM_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10BM_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 100 BM
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_BM_train, traits_BM_test = traits_BM[int(len(y)*.25):], traits_BM[:int(len(y)*.25)]

traits_BM_train=traits_BM_train.reshape((traits_BM_train.shape[0], (traits_BM_train.shape[1]*traits_BM_train.shape[2])))
traits_BM_test=traits_BM_test.reshape((traits_BM_test.shape[0], (traits_BM_test.shape[1]*traits_BM_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_BM_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_BM_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_BM_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_100BM_20SNPs.acc.mod')

In [ ]:
traits_OU = []
traits_OU = np.loadtxt("./traits/traits_OU.txt").reshape(30000,-1,100)
traits_OU = np.array(traits_OU)

scalers = {}
for i in range(traits_OU.shape[2]):
    scalers[i] = StandardScaler(copy=False)
    traits_OU[:, :, i] = scalers[i].fit_transform(traits_OU[:, :, i]) 

u1 = np.load("./trainingSims/Model_1sp.npz",mmap_mode='r')
u2 = np.load("./trainingSims/Model_2sp.npz",mmap_mode='r')
u3 = np.load("./trainingSims/Model_3sp.npz",mmap_mode='r')

X=np.concatenate((u1['Model_1sp'],u2['Model_2sp'],u3['Model_3sp']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(X):
    for idx,row in enumerate(array):
        if np.count_nonzero(row) > len(row)/2:
            X[arr][idx][X[arr][idx] == 1] = -1
            X[arr][idx][X[arr][idx] == 0] = 1
        else:
            X[arr][idx][X[arr][idx] == 0] = -1
            

y=[0 for i in range(len(u1['Model_1sp']))]
y.extend([1 for i in range(len(u2['Model_2sp']))])
y.extend([2 for i in range(len(u3['Model_3sp']))])
y = np.array(y)

print (len(X), len(y), len(traits_OU))
shf = list(range(len(X)))
shuffle(shf)

y = y[shf]
X = X[shf]
traits_OU = traits_OU[shf]

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
traits_OU_train, traits_OU_test = traits_OU[int(len(y)*.25):], traits_OU[:int(len(y)*.25)]


ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))
                                                                                                                                  # Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)
cnn = create_cnn(xtest)

In [ ]:
################################################################################################################################################
#100 OU, 1K SNPs
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model1.layers: print(layer.get_config(), layer.get_weights())
model1.save(filepath='./Trained_Models/Trained_Comb_Model_100OU_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#100 OU
################################################################################################################################################

!mlp = create_mlp(traits_OU_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

opt = SGD(learning_rate=0.001)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_OU_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_OU_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_100OU.acc.mod')

In [ ]:
################################################################################################################################################
#100 OU, 50 SNPs
################################################################################################################################################

xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model4 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model4.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model4.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model4.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model4.layers: print(layer.get_config(), layer.get_weights())
model4.save(filepath='./Trained_Models/Trained_Comb_Model_100OU_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 OU, 50 SNPs
################################################################################################################################################

traits_OU50=traits_OU[:,0:50,:]
traits_OU_train, traits_OU_test = traits_OU50[int(len(y)*.25):], traits_OU50[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50OU_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 OU
################################################################################################################################################
mlp = create_mlp(traits_OU_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()         
model3.fit(traits_OU_train, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(traits_OU_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_50OU.acc.mod')

In [ ]:
################################################################################################################################################
#50 OU, 20 SNPs
################################################################################################################################################

#X=np.concatenate((u1['simModel1'],u2['simModel2'],u4['simModel4'],u6['simModel6']),axis=0)
#X = X[shf]
traits_OU50=traits_OU[:,0:50,:]
traits_OU_train, traits_OU_test = traits_OU50[int(len(y)*.25):], traits_OU50[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50OU_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#10 OU
################################################################################################################################################
traits_OU10=traits_OU[:,0:10,:]
traits_OU_train, traits_OU_test = traits_OU10[int(len(y)*.25):], traits_OU10[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_OU_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_OU_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_10OU.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 10 OU
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

traits_OU10=traits_OU[:,0:10,:]
traits_OU_train, traits_OU_test = traits_OU10[int(len(y)*.25):], traits_OU10[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))
mlp = create_mlp(traits_OU_train)

combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10OU_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 10 OU
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_OU10=traits_OU[:,0:10,:]
traits_OU_train, traits_OU_test = traits_OU10[int(len(y)*.25):], traits_OU10[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))
mlp = create_mlp(traits_OU_train)
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10OU_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 50 OU
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_OU50=traits_OU[:,0:50,:]
traits_OU_train, traits_OU_test = traits_OU50[int(len(y)*.25):], traits_OU50[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
cnn = create_cnn(xtest)
mlp = create_mlp(traits_OU_train)
# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50OU_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50SNPS, 10 OU
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_OU10=traits_OU[:,0:10,:]
traits_OU_train, traits_OU_test = traits_OU10[int(len(y)*.25):], traits_OU10[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10OU_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 100 OU
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_OU_train, traits_OU_test = traits_OU[int(len(y)*.25):], traits_OU[:int(len(y)*.25)]

traits_OU_train=traits_OU_train.reshape((traits_OU_train.shape[0], (traits_OU_train.shape[1]*traits_OU_train.shape[2])))
traits_OU_test=traits_OU_test.reshape((traits_OU_test.shape[0], (traits_OU_test.shape[1]*traits_OU_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_OU_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_OU_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_OU_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_100OU_20SNPs.acc.mod')

In [ ]:
traits_disc = []
traits_disc = np.loadtxt("./traits/traits_disc.txt").reshape(30000,-1,100)
traits_disc = np.array(traits_disc)

u1 = np.load("./trainingSims/Model_1sp.npz",mmap_mode='r')
u2 = np.load("./trainingSims/Model_2sp.npz",mmap_mode='r')
u3 = np.load("./trainingSims/Model_3sp.npz",mmap_mode='r')

X=np.concatenate((u1['Model_1sp'],u2['Model_2sp'],u3['Model_3sp']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(X):
    for idx,row in enumerate(array):
        if np.count_nonzero(row) > len(row)/2:
            X[arr][idx][X[arr][idx] == 1] = -1
            X[arr][idx][X[arr][idx] == 0] = 1
        else:
            X[arr][idx][X[arr][idx] == 0] = -1
            

y=[0 for i in range(len(u1['Model_1sp']))]
y.extend([1 for i in range(len(u2['Model_2sp']))])
y.extend([2 for i in range(len(u3['Model_3sp']))])
y = np.array(y)

print (len(X), len(y), len(traits_disc))
shf = list(range(len(X)))
shuffle(shf)

y = y[shf]
X = X[shf]
traits_disc = traits_disc[shf]

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]

ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
traits_disc_train, traits_disc_test = traits_disc[int(len(y)*.25):], traits_disc[:int(len(y)*.25)]
traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)
cnn = create_cnn(xtest)

In [ ]:
################################################################################################################################################
#100 discrete, 1K SNPs
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)
model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model1.layers: print(layer.get_config(), layer.get_weights())
model1.save(filepath='./Trained_Models/Trained_Comb_Model_100disc_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#100 Discrete
################################################################################################################################################
traits_disc_train, traits_disc_test = traits_disc[int(len(y)*.25):], traits_disc[:int(len(y)*.25)]
traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

mlp = create_mlp(traits_disc_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

opt = SGD(learning_rate=0.001)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_disc_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_disc_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_100disc.acc.mod')

In [ ]:
################################################################################################################################################
#100 discrete, 50 SNPs
################################################################################################################################################

xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model4 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model4.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model4.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model4.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model4.layers: print(layer.get_config(), layer.get_weights())
model4.save(filepath='./Trained_Models/Trained_Comb_Model_100disc_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 disc, 50 SNPs
################################################################################################################################################
traits_disc50=traits_disc[:,0:50,:]
traits_disc_train, traits_disc_test = traits_disc50[int(len(y)*.25):], traits_disc50[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50disc_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50 discrete
################################################################################################################################################
traits_disc50=traits_disc[:,0:50,:]
traits_disc_train, traits_disc_test = traits_disc50[int(len(y)*.25):], traits_disc50[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())
          
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_disc_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_disc_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_50disc.acc.mod')


In [ ]:
################################################################################################################################################
#50 discrete, 20 SNPs
################################################################################################################################################
traits_disc50=traits_disc[:,0:50,:]
traits_disc_train, traits_disc_test = traits_disc50[int(len(y)*.25):], traits_disc50[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50disc_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#10 discrete
################################################################################################################################################
traits_disc10=traits_disc[:,0:10,:]
traits_disc_train, traits_disc_test = traits_disc10[int(len(y)*.25):], traits_disc10[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model3.fit(traits_disc_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_disc_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model3.layers: print(layer.get_config(), layer.get_weights())
model3.save(filepath='./Trained_Models/Trained_Traits_Model_10disc.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 10 discrete
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

mlp = create_mlp(traits_disc_train)

combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)

# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10disc_20SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 10 discrete
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_disc10=traits_disc[:,0:10,:]
traits_disc_train, traits_disc_test = traits_disc10[int(len(y)*.25):], traits_disc10[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
#xtest=xtest[:,0:20,:]
#xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)
mlp = create_mlp(traits_disc_train)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = SGD(learning_rate=0.001)
model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10disc_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#1KSNPS, 50 discrete
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_disc50=traits_disc[:,0:50,:]
traits_disc_train, traits_disc_test = traits_disc50[int(len(y)*.25):], traits_disc50[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)
# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_50disc_1KSNPs.acc.mod')

In [ ]:
################################################################################################################################################
#50SNPS, 10 discrete
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
traits_disc10=traits_disc[:,0:10,:]
traits_disc_train, traits_disc_test = traits_disc10[int(len(y)*.25):], traits_disc10[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:50,:]
xtrain=xtrain[:,0:50,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_10disc_50SNPs.acc.mod')

In [ ]:
################################################################################################################################################
#20SNPS, 100 disc
################################################################################################################################################xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]

traits_disc_train, traits_disc_test = traits_disc[int(len(y)*.25):], traits_disc[:int(len(y)*.25)]

traits_disc_train=traits_disc_train.reshape((traits_disc_train.shape[0], (traits_disc_train.shape[1]*traits_disc_train.shape[2])))
traits_disc_test=traits_disc_test.reshape((traits_disc_test.shape[0], (traits_disc_test.shape[1]*traits_disc_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_disc_train)

xtrain, xtest = X[int(len(y)*.25):], X[:int(len(y)*.25)]
xtest=xtest[:,0:20,:]
xtrain=xtrain[:,0:20,:]
cnn = create_cnn(xtest)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one softmax)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model6 = Model(inputs=[mlp.input, cnn.input], outputs=x)

model6.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model6.summary())
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=150, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model6.fit([traits_disc_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_disc_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

for layer in model6.layers: print(layer.get_config(), layer.get_weights())
model6.save(filepath='./Trained_Models/Trained_Comb_Model_100disc_20SNPs.acc.mod')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from keras.models import load_model
from sklearn.metrics import confusion_matrix


model1 = load_model('./Trained_Models/Trained_Traits_Model_100BM.acc.mod')
model2 = load_model('./Trained_Models/Trained_Traits_Model_50BM.acc.mod')
model3 = load_model('./Trained_Models/Trained_Traits_Model_10BM.acc.mod')
model4 = load_model('./Trained_Models/Trained_Traits_Model_100OU.acc.mod')
model5 = load_model('./Trained_Models/Trained_Traits_Model_50OU.acc.mod')
model6 = load_model('./Trained_Models/Trained_Traits_Model_10OU.acc.mod')
model7 = load_model('./Trained_Models/Trained_Traits_Model_100disc.acc.mod')
model8 = load_model('./Trained_Models/Trained_Traits_Model_50disc.acc.mod')
model9 = load_model('./Trained_Models/Trained_Traits_Model_10disc.acc.mod')
model10 = load_model('./Trained_Models/Trained_CNN_Model_1KSNPs.acc.mod')
model11 = load_model('./Trained_Models/Trained_CNN_Model_50SNPs.acc.mod')
model12 = load_model('./Trained_Models/Trained_CNN_Model_20SNPs.acc.mod')
model13 = load_model('./Trained_Models/Trained_Comb_Model_100BM_1KSNPs.acc.mod')
model14 = load_model('./Trained_Models/Trained_Comb_Model_50BM_1KSNPs.acc.mod')
model15 = load_model('./Trained_Models/Trained_Comb_Model_10BM_1KSNPs.acc.mod')
model16 = load_model('./Trained_Models/Trained_Comb_Model_100BM_50SNPs.acc.mod')
model17 = load_model('./Trained_Models/Trained_Comb_Model_50BM_50SNPs.acc.mod')
model18 = load_model('./Trained_Models/Trained_Comb_Model_10BM_50SNPs.acc.mod')
model19 = load_model('./Trained_Models/Trained_Comb_Model_100BM_20SNPs.acc.mod')
model20 = load_model('./Trained_Models/Trained_Comb_Model_50BM_20SNPs.acc.mod')
model21 = load_model('./Trained_Models/Trained_Comb_Model_10BM_20SNPs.acc.mod')
model22 = load_model('./Trained_Models/Trained_Comb_Model_100OU_1KSNPs.acc.mod')
model23 = load_model('./Trained_Models/Trained_Comb_Model_50OU_1KSNPs.acc.mod')
model24 = load_model('./Trained_Models/Trained_Comb_Model_10OU_1KSNPs.acc.mod')
model25 = load_model('./Trained_Models/Trained_Comb_Model_100OU_50SNPs.acc.mod')
model26 = load_model('./Trained_Models/Trained_Comb_Model_50OU_50SNPs.acc.mod')
model27 = load_model('./Trained_Models/Trained_Comb_Model_10OU_50SNPs.acc.mod')
model28 = load_model('./Trained_Models/Trained_Comb_Model_100OU_20SNPs.acc.mod')
model29 = load_model('./Trained_Models/Trained_Comb_Model_50OU_20SNPs.acc.mod')
model30 = load_model('./Trained_Models/Trained_Comb_Model_10OU_20SNPs.acc.mod')
model31 = load_model('./Trained_Models/Trained_Comb_Model_100disc_1KSNPs.acc.mod')
model32 = load_model('./Trained_Models/Trained_Comb_Model_50disc_1KSNPs.acc.mod')
model33 = load_model('./Trained_Models/Trained_Comb_Model_10disc_1KSNPs.acc.mod')
model34 = load_model('./Trained_Models/Trained_Comb_Model_100disc_50SNPs.acc.mod')
model35 = load_model('./Trained_Models/Trained_Comb_Model_50disc_50SNPs.acc.mod')
model36 = load_model('./Trained_Models/Trained_Comb_Model_10disc_50SNPs.acc.mod')
model37 = load_model('./Trained_Models/Trained_Comb_Model_100disc_20SNPs.acc.mod')
model38 = load_model('./Trained_Models/Trained_Comb_Model_50disc_20SNPs.acc.mod')
model39 = load_model('./Trained_Models/Trained_Comb_Model_10disc_20SNPs.acc.mod')

In [ ]:
#path = r'./testSims/traits/BM'
#all_files = glob.glob(path + "/*.txt")
#all_files.sort()
#all_files.sort(key=os.path.getmtime)
#s_all_files=all_files[0:2000]+all_files[3000:4000]+all_files[5000:6000]
#s_all_files=all_files[0:2000]+all_files[3000:5000]

traits_BM = []
#traits_BM = np.stack([np.loadtxt(f) for f in all_files])
traits_BM = np.loadtxt("./testSims/traits/traits_BM.txt").reshape(3000,-1,100)
traits_BM = np.array(traits_BM)

scalers = {}
for i in range(traits_BM.shape[2]):
    #scalers[i] = MinMaxScaler(feature_range=(-1, 1))
    scalers[i] = StandardScaler(copy=False)
    traits_BM[:, :, i] = scalers[i].fit_transform(traits_BM[:, :, i]) 
    

traits_BM10=traits_BM[:,0:10,:]
traits_BM50=traits_BM[:,0:50,:]

#path = r'./testSims/traits/OU'
#all_files = glob.glob(path + "/*.txt")
#all_files.sort()
#all_files.sort(key=os.path.getmtime)
#s_all_files=all_files[0:2000]+all_files[3000:4000]+all_files[5000:6000]
#s_all_files=all_files[0:2000]+all_files[3000:5000]

traits_OU = []
traits_OU = np.loadtxt("./testSims/traits/traits_OU.txt").reshape(3000,-1,100)
traits_OU = np.array(traits_OU)

scalers = {}
for i in range(traits_OU.shape[2]):
    #scalers[i] = MinMaxScaler(feature_range=(-1, 1))
    scalers[i] = StandardScaler(copy=False)
    traits_OU[:, :, i] = scalers[i].fit_transform(traits_OU[:, :, i]) 

traits_OU10=traits_OU[:,0:10,:]
traits_OU50=traits_OU[:,0:50,:]

#path = r'./testSims/traits/discrete'
#all_files = glob.glob(path + "/*.txt")
#all_files.sort()
#all_files.sort(key=os.path.getmtime)
#s_all_files=all_files[0:2000]+all_files[3000:4000]+all_files[5000:6000]
#s_all_files=all_files[0:2000]+all_files[3000:5000]

traits_disc = []
traits_disc = np.loadtxt("./testSims/traits/traits_disc.txt").reshape(3000,-1,100)
traits_disc = np.array(traits_disc)
traits_disc10=traits_disc[:,0:10,:]
traits_disc50=traits_disc[:,0:50,:]

u1 = np.load("./testSims/Model_1sp.npz",mmap_mode='r')
u2 = np.load("./testSims/Model_2sp.npz",mmap_mode='r')
u3 = np.load("./testSims/Model_3sp.npz",mmap_mode='r')

xtest=np.concatenate((u1['Model_1sp'],u2['Model_2sp'],u3['Model_3sp']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(xtest):
    for idx,row in enumerate(array):
        if np.count_nonzero(row) > len(row)/2:
            xtest[arr][idx][xtest[arr][idx] == 1] = -1
            xtest[arr][idx][xtest[arr][idx] == 0] = 1
        else:
            xtest[arr][idx][xtest[arr][idx] == 0] = -1
            
ytest=[0 for i in range(len(u1['Model_1sp']))]
ytest.extend([1 for i in range(len(u2['Model_2sp']))])
ytest.extend([2 for i in range(len(u3['Model_3sp']))])
ytest = np.array(ytest)

#ytest = np_utils.to_categorical(ytest, num_classes)

In [ ]:
#here's the confusion matrix function
def makeConfusionMatrixHeatmap(data, title, trueClassOrderLs, predictedClassOrderLs, ax):
    data = np.array(data)
    data = normalize(data, axis=1, norm='l1')
    heatmap = ax.pcolor(data, cmap=plt.cm.Blues, vmin=0.0, vmax=1.0)

    for i in range(len(predictedClassOrderLs)):
        for j in reversed(range(len(trueClassOrderLs))):
            val = 100*data[j, i]
            if val > 50:
                c = '0.9'
            else:
                c = 'black'
            ax.text(i + 0.5, j + 0.5, '%.2f%%' % val, horizontalalignment='center', verticalalignment='center', color=c, fontsize=9)

    cbar = plt.colorbar(heatmap, cmap=plt.cm.Blues, ax=ax)
    cbar.set_label("Fraction of simulations assigned to class", rotation=270, labelpad=20, fontsize=11)

    # put the major ticks at the middle of each cell
    ax.set_xticks(np.arange(data.shape[1]) + 0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[0]) + 0.5, minor=False)
    ax.axis('tight')
    ax.set_title(title)

    #labels
    ax.set_xticklabels(predictedClassOrderLs, minor=False, fontsize=9, rotation=45)
    ax.set_yticklabels(reversed(trueClassOrderLs), minor=False, fontsize=9)
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    


In [ ]:
#now the actual work
#first get the predictions
pred = model1.predict(traits_BM.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()
classOrderLs=['one','two','three']

#now do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model2.predict(traits_BM50.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
print (confusion_matrix(ytest, pred_cat))
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

#now do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

#get the predictions
pred = model3.predict(traits_BM10.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model4.predict(traits_OU.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()
classOrderLs=['one','two','three']

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model5.predict(traits_OU50.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
print (confusion_matrix(ytest, pred_cat))
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model6.predict(traits_OU10.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model7.predict(traits_disc.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

#now the actual work
#first get the predictions
pred = model8.predict(traits_disc50.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

#now do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model9.predict(traits_disc10.reshape(3000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model10.predict(xtest)
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model11.predict(xtest[:,0:50,:])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model12.predict(xtest[:,0:20,:])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model13.predict([traits_BM.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model14.predict([traits_BM50.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model15.predict([traits_BM10.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model16.predict([traits_BM.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model17.predict([traits_BM50.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model18.predict([traits_BM10.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model19.predict([traits_BM.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model20.predict([traits_BM50.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model21.predict([traits_BM10.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model22.predict([traits_OU.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model23.predict([traits_OU50.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model24.predict([traits_OU10.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model25.predict([traits_OU.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model26.predict([traits_OU50.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model27.predict([traits_OU10.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model28.predict([traits_OU.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model29.predict([traits_OU50.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model30.predict([traits_OU10.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model31.predict([traits_disc.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()


# get the predictions
pred = model32.predict([traits_disc50.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()


# get the predictions
pred = model33.predict([traits_disc10.reshape(3000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model34.predict([traits_disc.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model35.predict([traits_disc50.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model36.predict([traits_disc10.reshape(3000,-1), xtest[:,0:50,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model37.predict([traits_disc.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model38.predict([traits_disc50.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()

# get the predictions
pred = model39.predict([traits_disc10.reshape(3000,-1), xtest[:,0:20,:]])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]

counts=[[0.,0.,0.],[0.,0.,0.],[0.,0.,0.]]
for i in range(len(ytest_cat)):
    counts[ytest[i]][pred_cat[i]] += 1
counts.reverse()

# do the plotting
fig,ax= plt.subplots(1,1)
makeConfusionMatrixHeatmap(counts, "Confusion matrix", classOrderLs, classOrderLs, ax)
plt.show()